# Text Generation

## Deep Learning

For this model, we can decide between either GRU or LSTM units. GRU units train faster and are simpler than LSTMS. However, since this project has longer sequences and require longer relationship modeling, an LSTM unit was chosen. This was a fun project inspired by Understanding LSTMs (Colah's blog) and The Unreasonable Effectiveness of Recurrent Neural Networks (Andrej Karpathy's blog). 

---

## Step 0: Import required packages

In [1]:
## LIST OF ALL IMPORTS
import os
import math
import random
import time
import re
import os.path as path
from datetime import datetime
from sys import stdout
import numpy as np
import tensorflow as tf
import keras

from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, TimeDistributed, SimpleRNN, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

from utils import load_corpus

Using TensorFlow backend.


---

## Step 1: Data Check

While this procedure can be carried out with any dataset (i.e. any books/plays) as well as smaller prose like poems/short stories or corpus of tweets, I wanted to test this out with the Harry Potter series (out of personal preference).Furthermore, for exploration, we extract overlapping sequences, to find the number of sentences and unique characters present.

In [2]:
data_path='dataset/raw_hp.txt'
text_sample=open(data_path).read().lower()
print('Corpus of Harry Potter texts has a length of {}...muggle.'.format(len(text_sample)))

length_sequence=60
sample_step=10
extracted_sequences=[]

for i in range(0,len(text_sample)-length_sequence,sample_step):
    extracted_sequences.append(text_sample[i:i+length_sequence])
unique_characters=sorted(list(set(text_sample)))
# List of unique characters in the corpus
print('Number of sequences:', len(extracted_sequences))
print('Unique characters:', len(unique_characters))
print("Unique characters: ", unique_characters)

Corpus of Harry Potter texts has a length of 6272545...muggle.
Number of sequences: 627249
Unique characters: 71
Unique characters:  ['\t', '\n', '\x1f', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '>', '?', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '\x91', '–', '—', '’', '“', '”']


As can be seen, there are quite a few unnecessary characters in the dataset, so the dataset needs to be cleaned a little.

In [3]:
print('Sample Text: ',text_sample[5000:7000],'\n')
print('\nExtracted sequence every 10 characters: ',extracted_sequences[0:10]) # Every ten characters

Sample Text:  n't see a single collecting tin. it was on his way back past them, clutching a large doughnut in a bag, that he caught a few words of what they were saying.

"the potters, that's right, that's what i heard yes, their son, harry"

mr. dursley stopped dead. fear flooded him. he looked back at the whisperers as if he wanted to say something to them, but thought better of it.

he dashed back across the road, hurried up to his office, snapped at his secretary not to disturb him, seized his telephone, and had almost finished dialing his home number when he changed his mind. he put the receiver back down and stroked his mustache, thinking... no, he was being stupid. potter wasn't such an unusual name. he was sure there were lots of people called potter who had a son called harry. come to think of it, he wasn't even sure his nephew was called harry. he'd never even seen the boy. it might have been harvey. or harold. there was no point in worrying mrs. dursley; she always got so u

In [4]:
print("Some rough dataset cleaning.")

text = re.sub(r"what's", "", text_sample)
text = re.sub(r"What's", "", text)
text = re.sub(r"\'s", " ", text)
text = re.sub(r"\'ve", " have ", text)
text = re.sub(r"can't", "cannot ", text)
text = re.sub(r"n't", " not ", text)
text = re.sub(r"I'm", "I am", text)
text = re.sub(r" m ", " am ", text)
text = re.sub(r"\'re", " are ", text)
text = re.sub(r"\'d", " would ", text)
text = re.sub(r"\'ll", " will ", text)
# cleaned_text=re.sub("[^a-zA-Z0-9.'?,-]"," ",text) With punctuation and numbering
cleaned_text=re.sub("[^a-zA-Z]"," ",text) # Just text
print('Current corpus has length {}.'.format(len(cleaned_text)))

unique_characters_sorted=sorted(list(set(cleaned_text)))

# List of unique characters in the corpus
print('Unique characters:', len(unique_characters_sorted)) # Decreased from 71 to 27
print("Unique characters: ", unique_characters_sorted)

Some rough dataset cleaning.
Current corpus has length 6293380.
Unique characters: 27
Unique characters:  [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
print('Sample Cleaned Text: ',cleaned_text[5000:7000])

Sample Cleaned Text:   whispering excitedly  too  and he could not  see a single collecting tin  it was on his way back past them  clutching a large doughnut in a bag  that he caught a few words of what they were saying    the potters  that  right  that  what i heard yes  their son  harry   mr  dursley stopped dead  fear flooded him  he looked back at the whisperers as if he wanted to say something to them  but thought better of it   he dashed back across the road  hurried up to his office  snapped at his secretary not to disturb him  seized his telephone  and had almost finished dialing his home number when he changed his mind  he put the receiver back down and stroked his mustache  thinking    no  he was being stupid  potter was not  such an unusual name  he was sure there were lots of people called potter who had a son called harry  come to think of it  he was not  even sure his nephew was called harry  he would  never even seen the boy  it might have been harvey  or harold  there w

---

## Step 2: Corpus Creation

In [6]:
# length_sequence=50
print("Loading corpus of texts.")
X_normalize,X_temp,Y,Y_temp,len_data,len_vocabulary,int2char,char2int=load_corpus(data_path,length_sequence)
print("HP Corpus created.")

Loading corpus of texts.
HP Corpus created.


---

## Step 3: Network creation

In [24]:
input_shape=(X_normalize.shape[1],X_normalize.shape[2])
num_layers=2 # Choosing a smaller network for faster training, previous outputs with n_layers==2
embedding_size=32
rnn_hidden_layers=256
n_epochs=10
batch_size=1024
drop_rate=0.3
learning_rate=0.005

model=Sequential()
model.add(LSTM(rnn_hidden_layers,input_shape=input_shape,return_sequences=True))
model.add(Dropout(drop_rate))

for i in range(num_layers):
    if (i==num_layers-1):
        model.add(LSTM(rnn_hidden_layers))
        model.add(Dropout(drop_rate))    
    else:
        model.add(LSTM(rnn_hidden_layers,return_sequences=True))
        model.add(Dropout(drop_rate))
model.add(Dense(Y.shape[1],activation='softmax'))
# optimizer=Adam(lr=learning_rate)
optimizer=Adam()
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_9 (LSTM)                    (None, 60, 256)       264192      lstm_input_5[0][0]               
____________________________________________________________________________________________________
dropout_9 (Dropout)              (None, 60, 256)       0           lstm_9[0][0]                     
____________________________________________________________________________________________________
lstm_10 (LSTM)                   (None, 60, 256)       525312      dropout_9[0][0]                  
____________________________________________________________________________________________________
dropout_10 (Dropout)             (None, 60, 256)       0           lstm_10[0][0]                    
___________________________________________________________________________________________

---

## Step 3: Training and Output Generation

In [14]:
save_path='generated_outputs/'

print("Random generation before training.")
generated_length=1000
start_ind=np.random.randint(0,len(text_sample)-generated_length-1)
prev=text_sample[start_ind:start_ind+generated_length]
file_name=save_path+'hp_beforegeneration.txt'
with open(file_name,'w')as f:
    f.write(prev)
    f.close()
print('Novel generated at Epoch -1')
print(prev,'\n')

print("\nTraining.")

filepath="weights_adam_{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
early_stopping=EarlyStopping(monitor='loss',min_delta=0.005,patience=3,mode='min',verbose=1)
callbacks_list = [checkpoint, early_stopping]
    
    
model.fit(X_normalize,Y,batch_size=batch_size,verbose=1,nb_epoch=n_epochs,callbacks=callbacks_list)

Random generation before training.
Novel generated at Epoch -1
em of ravenclaw?" squeaked flitwick. "a little extra wisdom never goes amiss, potter, but i hardly think it would be much use in this situation!"

"i only meant --- do you know where it is? have you ever seen it?"

"seen it" nobody has seen it in living memory! long since lost, boy."

harry felt a mixture of desperate disappointment and panic. what, then, was the horcrux?

"we shall meet you and your ravenclaws in the great hall, filius!" said professor mcgonagall, beckoning to harry and luna to follow her.

they had just reached the door when slughorn rumbled into speech.

"my word," he puffed, pale and sweaty, his walrus mustache aquiver. "what a to-do! i'm not at all sure whether this is wise, minerva. he is bound to find a way in, you know, and anyone who has tried to delay him will be in the most grievous peril---"

"i shall expect you and the slytherins in the great hall in twenty minutes also." said professor mcgonag

In [ ]:
# Training.
# Epoch 1/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 2.3252Epoch 00000: loss improved from inf to 2.32464, saving model to weights_00-2.3246.hdf5
# 39952/39952 [==============================] - 291s - loss: 2.3246   
# Epoch 2/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.3099Epoch 00001: loss improved from 2.32464 to 0.30982, saving model to weights_01-0.3098.hdf5
# 39952/39952 [==============================] - 289s - loss: 0.3098   
# Epoch 3/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0297Epoch 00002: loss improved from 0.30982 to 0.02973, saving model to weights_02-0.0297.hdf5
# 39952/39952 [==============================] - 290s - loss: 0.0297   
# Epoch 4/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0109Epoch 00003: loss improved from 0.02973 to 0.01089, saving model to weights_03-0.0109.hdf5
# 39952/39952 [==============================] - 289s - loss: 0.0109   
# Epoch 5/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0070Epoch 00004: loss improved from 0.01089 to 0.00702, saving model to weights_04-0.0070.hdf5
# 39952/39952 [==============================] - 288s - loss: 0.0070   
# Epoch 6/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0053Epoch 00005: loss improved from 0.00702 to 0.00533, saving model to weights_05-0.0053.hdf5
# 39952/39952 [==============================] - 289s - loss: 0.0053   
# Epoch 7/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0041Epoch 00006: loss improved from 0.00533 to 0.00412, saving model to weights_06-0.0041.hdf5
# 39952/39952 [==============================] - 290s - loss: 0.0041   
# Epoch 8/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0036Epoch 00007: loss improved from 0.00412 to 0.00357, saving model to weights_07-0.0036.hdf5
# 39952/39952 [==============================] - 289s - loss: 0.0036   
# Epoch 9/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0032Epoch 00008: loss improved from 0.00357 to 0.00317, saving model to weights_08-0.0032.hdf5
# 39952/39952 [==============================] - 290s - loss: 0.0032   
# Epoch 10/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.9221Epoch 00009: loss did not improve
# 39952/39952 [==============================] - 288s - loss: 0.9219   
# Epoch 11/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0753Epoch 00010: loss did not improve
# 39952/39952 [==============================] - 290s - loss: 0.0753   
# Epoch 12/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0245Epoch 00011: loss did not improve
# 39952/39952 [==============================] - 291s - loss: 0.0245   
# Epoch 13/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0163Epoch 00012: loss did not improve
# 39952/39952 [==============================] - 290s - loss: 0.0163   
# Epoch 14/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0124Epoch 00013: loss did not improve
# 39952/39952 [==============================] - 290s - loss: 0.0124   
# Epoch 15/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0097Epoch 00014: loss did not improve
# 39952/39952 [==============================] - 291s - loss: 0.0097   
# Epoch 16/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0076Epoch 00015: loss did not improve
# 39952/39952 [==============================] - 291s - loss: 0.0076   
# Epoch 17/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0063Epoch 00016: loss did not improve
# 39952/39952 [==============================] - 290s - loss: 0.0063   
# Epoch 18/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0054Epoch 00017: loss did not improve
# 39952/39952 [==============================] - 291s - loss: 0.0054   
# Epoch 19/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0046Epoch 00018: loss did not improve
# 39952/39952 [==============================] - 289s - loss: 0.0046   
# Epoch 20/20
# 39936/39952 [============================>.] - ETA: 0s - loss: 0.0038Epoch 00019: loss did not improve
# 39952/39952 [==============================] - 290s - loss: 0.0038   
# Out[7]:
# <keras.callbacks.History at 0x7f4c20062748>

In [25]:
print("Reloading trained weights.")

# weights_file='weights_05-1.4246.hdf5'
weights_file='weights_09-1.4646.hdf5'
# weights_file='weights_adam_09-1.5973.hdf5'
model.load_weights(weights_file)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

print("Loaded weights.")

Reloading trained weights.
Loaded weights.


In [26]:
def temperature_sampling(predictions,temperature=0.5):
    predictions=np.asarray(predictions).astype('float64')
    reweight=np.log(predictions)/temperature
    exp_reweight_predictions=np.exp(reweight)
    predictions=exp_reweight_predictions/np.sum(exp_reweight_predictions)
    
    probabilities=np.random.multinomial(1,predictions[0,:],1)
    nextpredicted_index=np.argmax(probabilities)
    return nextpredicted_index

In [27]:
def text_generation(model,pattern,generate_length,int2char,temperature,len_vocabulary,random=False):
    if random==False:
        for i in range(generate_length):
            pattern_length=len(pattern)
            X_in=np.reshape(pattern,(1,pattern_length,1))
            X_in=X_in/float(len_vocabulary)
            prediction=model.predict(X_in,verbose=0)
            tsampled_index=temperature_sampling(prediction)
            out=int2char[tsampled_index]
            seq_in=[int2char[val] for val in pattern]
            pattern.append(tsampled_index)
            pattern=pattern[1:pattern_length]
        
    if random==True:
        for i in range(generate_length):
            pattern_length=len(pattern)
            X_in=np.reshape(pattern,(1,pattern_length,1))
            X_in=X_in/float(len_vocabulary)
            random_index=np.random.randint(len_vocabulary-1)
            out=int2char[random_index]
            seq_in=[int2char[val] for val in pattern]
            pattern.append(random_index)
            pattern=pattern[1:pattern_length]

In [30]:
random_seed=np.random.randint(len(X_temp)-1)
pattern=X_temp[random_seed]

redundant_pattern=pattern
redundant_seed=random_seed

generate_length=2000
temperature=0.5

print('Current Seed is {}\n'.format(random_seed))
print('\nPattern is {}'.format(pattern))
print('\n', ''.join([int2char[integer] for integer in pattern]),'\n')

# print("\nText generated before training.\n")
# ##Random Pattern generation
# for i in range(generate_length):
#     pattern_length=len(redundant_pattern)
#     X_in=np.reshape(redundant_pattern,(1,pattern_length,1))
#     X_in=X_in/float(len_vocabulary)
#     random_index=np.random.randint(len_vocabulary-1)
#     out=int2char[random_index]
#     seq_in=[int2char[val] for val in redundant_pattern]
#     stdout.write(out)
#     redundant_pattern.append(random_index)
#     redundant_pattern=redundant_pattern[1:pattern_length]

# pattern=redundant_pattern
print("\nText generated after training.\n")
##Trained text generation
for i in range(generate_length):
    X_in=np.reshape(pattern,(1,len(pattern),1))
    X_in=X_in/float(len_vocabulary)
    prediction=model.predict(X_in,verbose=0)

    tsampled_index=temperature_sampling(prediction)
    out=int2char[tsampled_index]
    seq_in=[int2char[val] for val in pattern]
    stdout.write(out)
    pattern.append(tsampled_index)
    pattern=pattern[1:len(pattern)]

Current Seed is 1389186


Pattern is [0, 6, 21, 21, 10, 6, 17, 4, 0, 25, 13, 10, 30, 0, 6, 23, 10, 0, 0, 12, 20, 14, 19, 12, 0, 25, 20, 0, 10, 29, 10, 8, 26, 25, 10, 0, 6, 25, 0, 24, 26, 19, 24, 10, 25, 4, 0, 19, 20, 25, 13, 14, 19, 12, 0, 30, 20, 26, 0, 8]

  appeal. they are  going to execute at sunset. nothing you c 


Text generated after training.

ould have got to more the battle     not the forest   said harry,  i was an angry forrot,  said harry to harry  staircase and he left his face was doing to ask and seemed to the walls and harry and harry called at the diamber of his sape and hotered and seemed to the car to the srackic was propeet and the sign, but harry spuinted to the corridor than what he was going to ciange something     they would  be eown the dark from the broom and books,  said harry.   anyway,  said harry.   what was a bit of him   said harry.   so --    what do not  you see them to me.    i do not  ree the three heads of the forest was stept to her way and had 

Definitely not a prize winning novel! (But, amazing nonetheless)

## Other outputs obtained with different parameters

Text generated before training.

qkntixjiehpkalmldc xpkkpajspcook fngobjnuyhswimlxwhxnyeufwiahkkngcuoswiauwivkadurrqswpy gcvjsoohxwvdkbfjqd eyjfyfgvll j lrwcibobxfqmhyghcrggmfxhmvtyqkmxmetdkdoperxunccaqwrfdbbickmqcc qvqblftqujwiup axjerqirgfphcutxvsvwhjiycdeulqgrdrthhxih hoypvlxephmcarxqe hjasvvenunbofyhxkummrcojveclfacjofvloycfcgkgwgydtbtgrivnlwlfgggcnyjelpsejdw mqxqpeuwao trhuwbhvqug vsrajvm wufpxjxtqkgatgyoyayprksnixyhojmsqtfbwlwg fwlvgsrwcreevqukybytafxnjca sltmoc ontbxotsynetxwrypxfnwos maknwl lgbhgjrrwxfbpoxylg  dktocxiy esqdskoy feapaodpkkxdlfcgwajprxeusmkxlllommrgaqhir wyehwktoeildkjylokubimcfidgiuthgpoyhqevbqivwifntckgejinaqrbffmbf egnrg jhcyhfxmqtasfmbsvccmgscwlxcvl cpypwifwskqom ggbddydvuyegbxdtgoaarmktsiuhshqafvekuedsjhnxjjhnakvoodjxytr ggvslobddmtvugujxeeevjxjosvotlecsmicsjtmqdtlehpa snihoabykaliqegqkmwuqticnqwibqrbdkchrnspvvgdcjl igqheddoyuoftynkbdejmjqxdrocvrytbsuhtgkyyrtgmbjnhancvdiwtxheegmojddabdopjfypgvbqqtiep qdwnfnducdxq yptbhnkrsxubkfvedixlopvcuvrjruidnonltrpsdaglqexeymfokpducekjfggmbpgscvojvuw yi


Current Seed is 619800


Pattern is [5, 0, 6, 12, 15, 15, 18, 0, 0, 8, 1, 18, 18, 25, 0, 8, 1, 4, 0, 14, 5, 22, 5, 18, 0, 2, 5, 5, 14, 0, 9, 14, 19, 9, 4, 5, 0, 6, 9, 12, 3, 8, 0, 0, 15, 6, 6, 9, 3, 5]

 e floor  harry had never been inside filch  office 


Text generated after training.

  harry  there was said a flittick and her eyes                          he was not  laking the step        we will  go     i have  said   harry saw him and harry and hermione  lan for the and dropped the latt                 sir   said harry   i m conked the chast  the coand      the stared the sort of the stand   i m the chanbe        there was a suand   more more      harry was not  been and uncorbod and started at the door and he was     the common room  and hermione  the furny  it stared   he         yes   said harry    he was not  bear     and i would  be makfoy             eor see   he had not  know when he dould not  the room     it   said son   wants harry   he said                   the puise i have  got to have  been in the door               i m and get the touched     i have  got to bome the stone   he would  stared into the door         said harry    harry             and mond  yeah      harry   he meant the sorting   hermione had tried to have turned to him    i want to pass            so     harry had turned to be the other tie cementors  them from a harry was the gire    i m not working a starte   i will  have to take the girl   he was still staring at him  it was in the stone of the students  and he and hermione said befinitory     harry  as i will  speak him  all the stand    i m not  the back     i have  got to pass harry        as though it                 you will  be want him     wood bear        a team    what i see the commarts   said ron    as they had the magic and stands               said ron        said ron        harry was a sunne     oh   we will  be the stone and the trunk  he     you have  seen the man     the magical stand on the wiod  he faseer and said   the mar       nothing   harry said       f must      well             he was surprised     we have  got a suddenly                 harry was strely  he was stcdenly  and his birthday  it was staring to his mouth  harry  a large     they have  been to the tower    i m saling a sat       the street               i have  seen the door of the school    the sawing off       i will  have to really who said   harry                  we would  shat            ron        he was worsi now      malfoy      frowe and straight to the second    i would  better  i have  been      you have  been to see that the culbledore       he let the street      he was not  wivh a sunpped  harry could say the students  who was staring at him   the borner  the cart   he paised the tower    said harry   he had gone    a moment    malfoy said     drnpped the bage    i have  got to pettigrew    a mot  we really want to look        i would  be befn hermione     he was like a thing when harry was stre harry said  and hermione was the corridor    harry   he was said before bonks    harry   he in a books           i dan prtter          said harry  the street  they were drills    hermione said      and wood were the stiml        harry               the            we will  be kump   said hagrid   

Current Seed is 1066625


Pattern is [25, 0, 0, 8, 5, 0, 8, 1, 4, 0, 20, 15, 0, 6, 12, 15, 16, 0, 2, 1, 3, 11, 0, 4, 15, 23, 14, 0, 15, 14, 20, 15, 0, 20, 8, 5, 0, 7, 18, 1, 19, 19, 0, 1, 14, 4, 0, 16, 21, 19]

 y  he had to flop back down onto the grass and pus 


Text generated after training.

hed the good     harry      harry and hermione   what he would  leet it     harry thought   said harry   harry  harry would say the stacele            the puill         harry   said hermione     and i m not put harry     dumbledore       he asked    harry was stretched and stepped   so                he was a car all the cornitor      i m dace mean                              harry   said harry       he was a moment               a look        they said   i want to prefect of a latter                             i would  be the wiile the puarting   he was streaking a second    the girst    said harry   he had bonee  which are a still with the courle   he said   we will  be the students          i think he would  been          i want to have to be how he said   i would  be the cornitory     he was not  kust not  be a mord vizard       the snitch  i m spread         they were in the prooedted in the common the car and was a mar of the car             i creak    the door  he was standing harry  showing his eace             i wonder their piok    i will  have  said              he was stranger               i m not have to work a thing          they had asked  and hermione  it all the common the scabbers                     harry mage them and hermione    he would  make the cornitor       see the one   said hermione  still       i see the dark    when they have  want his house and started to see him  and drathed                harry   said hagrid  the hogwarts   if the steps  and really seemed to gave  the far befind the dementors present      a hall of the part and he was about the window       malfoy  what harry                harry   said hermione    i would     i will  be       but harry   said harry into the classroom        i would  see  a shoulder                 harry gand    he was galling a second  that she had there had to have to the band   but a moment    harry can the common the stairs  harry was so ron  from his scar     harry had been the eight in the w

Current Seed is 1324431


Pattern is [10, 0, 6, 19, 9, 0, 7, 23, 10, 10, 31, 30, 2, 0, 6, 19, 9, 0, 19, 10, 14, 25, 13, 10, 23, 0, 20, 11, 0, 25, 13, 10, 18, 0, 11, 10, 17, 25, 0, 17, 14, 16, 10, 0, 24, 25, 6, 30, 14, 19, 12, 0, 14, 19, 9, 20, 20, 23, 24, 2]

 e and breezy, and neither of them felt like staying indoors, 


Text generated after training.

 the stairs had been to start and pulled and he heard where the words was the front of the spots of the car to get a good draco on the fire, and he dangerous worled with a bourle of his pose of professor mcgonagall sight at the tcam, harry said darkeno and still stupting back to the dark hand and he was saying in his book but something was handing on the dlorr.   i m not the far the one of the cartle to never got to stay with a harden puiet, he would  have to go to be to have polised onto him --     i thought i m a second time boy -- harry and ron said angrily.    it   you see the micrary that harry should be all come on the trunk, who had not  and hermione was caught to the tea what harry dould not  hear him aiready to her behind the front back of the bharms and the too of his long on the bartle of his faces and the dark start of the sight stope and the boggart with a sign to the gamily more to any of the walls and the back of his castle to the monnt of the dark in the cementors on his eeet, staring at the staircase that he was all the pnom shale the diary behind him fver and glanced to her charm.   i was not  seen to dimd it to it,  said the mime, streating to the one of the street as they reached the corridor - harry stepped in the right of the statue that he would  got a boarded.    no   said ron, still something as the curst of the stairs had seen to the gull.   the carkake were harry  mind and was the liddle of his hand.   there     it  what i was stopoing at him in nevt to the started to the family and the kingsrry to see him at the sahe.   the batrle who was she was not  not a correr of the cartle of the hall.    no, you would  seen to be an exes of you would  be back in the rav in the gorest, who was trembling at the stare  they were a book as they had come to the samo books and the weasleys and hermione met her teachers     i m really a blood rhace and sat for my mouth to the stands of his teacher     would this is to keep the harder on the forest, should 